- the goal of this project is to build a path planner that creates smooth, safe trajectories for the car to follow
- the highway track has other vehicles, all going different speeds, but approximately obeying the 50 MPH speed limit
- the car transmits its location, along with its sensor fusion data, which estimates the location of all the vehicles on the same side of the road

<img src="resources/car_traversing_path_points.gif"/>

#### Point Paths

- the path planner should output a list of $x$ and $y$ global map coordinates
- each pair of $x$ and $y$ coordinates is a point, and all of the points together form a trajectory
- you can use any number of points that you want, but the $x$ list should be the same length as the $y$ list
- every 20 ms the car moves to the next point on the list
- the car's new rotation becomes the line between the previous waypoint and the car's new location
- the car moves from point to point perfectly, so you don't have to worry about building a controller for this project
- the animation above shows how the car moves and rotates through a given list of points

#### Velocity

- the velocity of the car depends on the spacing of the points
- because the car moves to a new waypoint every 20ms, the larger the spacing between points, the faster the car will travel
- the speed goal is to have the car traveling at (but not above) the 50 MPH speed limit as often as possible
- but there will be times when traffic gets in the way

## Getting Started

- start by simply trying to move the car forward in a straight line at a constant 50 MPH velocity
- use the car's $(x, y)$ localization information and its heading direction to create a simple, straight path that is drawn directly in front of the car


- in `main.cpp`, instead of setting the speed directly, we pass `next_x_vals`, and `next_y_vals` to the simulator
- we will set the points 0.5 m apart
- since the car moves 50 times a second, a distance of 0.5m per move will create a velocity of 25 m/s
- 25 m/s is close to 50 MPH

```cpp
double dist_inc = 0.5;
for (int i = 0; i < 50; ++i) {
  next_x_vals.push_back(car_x+(dist_inc*i)*cos(deg2rad(car_yaw)));
  next_y_vals.push_back(car_y+(dist_inc*i)*sin(deg2rad(car_yaw)));
}
```

<img src="resources/car_in_line.gif"/>

- notice, however, that the car went from 0 MPH to 56 MPH in a single 20 ms frame, causing a spike in acceleration
- acceleration is calculated by comparing the rate of change of average speed over 0.2 second intervals
- in this case total acceleration at one point was as high as 75 m/s^2
- jerk was also very high
- the jerk is calculated as the average acceleration over 1 second intervals
- in order for the passenger to have an enjoyable ride both the jerk and the total acceleration should not exceed 10 m/s^2


- part of the total acceleration is the normal component, AccN which measures the centripetal acceleration from turning
- the tighter and faster a turn is made, the higher the AccN value will be
- in our simple test we were not turning at all, so the value of AccN was zero


- going forward, consider how to minimize total acceleration and jerk by gradually increasing and decreasing point path spacing based on the `car_speed` variable
- to get a better idea of how movement affects the acceleration of the car, click the Manual Mode check box in the top left of the simulator screen to drive the car around yourself

## More Complex Paths

- in the previous concept you moved the car forward in a straight line
- how about creating more interesting paths (e.g. circular path)?

<img src="resources/circle_path.gif"/>

```cpp
vector<double> next_x_vals;
vector<double> next_y_vals;

double pos_x;
double pos_y;
double angle;
int path_size = previous_path_x.size();

for (int i = 0; i < path_size; ++i) {
  next_x_vals.push_back(previous_path_x[i]);
  next_y_vals.push_back(previous_path_y[i]);
}

if (path_size == 0) {
  pos_x = car_x;
  pos_y = car_y;
  angle = deg2rad(car_yaw);
} else {
  pos_x = previous_path_x[path_size-1];
  pos_y = previous_path_y[path_size-1];

  double pos_x2 = previous_path_x[path_size-2];
  double pos_y2 = previous_path_y[path_size-2];
  angle = atan2(pos_y-pos_y2,pos_x-pos_x2);
}

double dist_inc = 0.5;
for (int i = 0; i < 50-path_size; ++i) {    
  next_x_vals.push_back(pos_x+(dist_inc)*cos(angle+(i+1)*(pi()/100)));
  next_y_vals.push_back(pos_y+(dist_inc)*sin(angle+(i+1)*(pi()/100)));
  pos_x += (dist_inc)*cos(angle+(i+1)*(pi()/100));
  pos_y += (dist_inc)*sin(angle+(i+1)*(pi()/100));
}

msgJson["next_x"] = next_x_vals;
msgJson["next_y"] = next_y_vals;
```


#### Using Previous Path Points

- the code snippet above builds a 50 point path, as in the previous experiment
- but this code snippet starts the new path with whatever previous path points were left over from the last cycle
- then we append new waypoints, until the new path has 50 total waypoints
- using information from the previous path ensures that there is a smooth transition from cycle to cycle
- but the more waypoints we use from the previous path, the less the new path will reflect dynamic changes in the environment.
- ideally, we might only use a few waypoints from the previous path and then generate the rest of the new path based on new data from the car's sensor fusion information


#### Timing

- the simulator runs a cycle every 20 ms (50 frames per second), but your C++ path planning program will provide a new path at least one 20 ms cycle behind
- the simulator will simply keep progressing down its last given path while it waits for a new generated path
- this means that using previous path data becomes even more important when higher latency is involved
- imagine, for instance, that there is a 500ms delay in sending a new path to the simulator
- as long as the new path incorporates a sufficient length of the previous path, the transition will still be smooth


- a concern, though, is how accurately we can predict other traffic 1-2 seconds into the future
- an advantage of newly generated paths is that they take into account the most up-to-date state of other traffic


#### Setting Point Paths with Latency

- as mentioned, your C++ path planner will at the very least be one cycle behind the simulator because the C++ program can't receive and send data on the same cycle
- as a result, any path that the simulator receives will be from the perspective of a previous cycle
- this might mean that by the time a new path reaches the simulator, the vehicle has already passed the first few waypoints on that path
- luckily you don't have to worry about this too much
- the simulator has built-in tools to deal with this timing difference
- the simulator actually expects the received path to be a little out of date compared to where the car is, and the simulator will consider which point on the received path is closest to the car and adjust appropriately

## Highway Map

- inside `data/highway_map.csv` there is a list of waypoints that go all the way around the track
- the track contains a total of 181 waypoints, with the last waypoint mapping back around to the first
- the waypoints are in the middle of the double-yellow dividing line in the center of the highway
- the track is 6945.554 meters around (about 4.32 miles)
- if the car averages near 50 MPH, then it should take a little more than 5 minutes for it to go all the way around the highway
- the highway has 6 lanes total - 3 heading in each direction
- each lane is 4 m wide and the car should only ever be in one of the 3 lanes on the right-hand side
- the car should always be inside a lane unless doing a lane change


#### Waypoint Data

- each waypoint has an (x,y) global map position, and a Frenet s value and Frenet d unit normal vector (split up into the x component, and the y component)
- the s value is the distance along the direction of the road
- the first waypoint has an s value of 0 because it is the starting point
- the d vector has a magnitude of 1 and points perpendicular to the road in the direction of the right-hand side of the road
- the d vector can be used to calculate lane positions


- for example, if you want to be in the left lane at some waypoint just add the waypoint's (x,y) coordinates with the d vector multiplied by 2
- since the lane is 4 m wide, the middle of the left lane (the lane closest to the double-yellow dividing line) is 2 m from the waypoint
- if you would like to be in the middle lane, add the waypoint's coordinates to the d vector multiplied by 6 = (2+4), since the center of the middle lane is 4 m from the center of the left lane, which is itself 2 m from the double-yellow dividing line and the waypoints


#### Converting Frenet Coordinates

- we have included a helper function, `getXY`, which takes in Frenet (s,d) coordinates and transforms them to (x,y) coordinates


#### Interpolating Points

- if you need to estimate the location of points between the known waypoints, you will need to "interpolate" the position of those points
- in previous lessons we looked at fitting polynomials to waypoints
- once you have a polynomial function, you can use it to interpolate the location of a new point
- there are also other methods you could use
- for example, Bezier curve fitting with control points, or spline fitting, which guarantees that the generated function passes through every point
- [here](http://kluge.in-chemnitz.de/opensource/spline/) is a great and easy to setup and use spline tool for C++, contained in just a single header file

## Sensor Fusion

- it's important that the car doesn't crash into any of the other vehicles on the road, all of which are moving at different speeds around the speed limit and can change lanes
- the `sensor_fusion` variable contains all the information about the cars on the right-hand side of the road
- the data format for each car is: [id, x, y, vx, vy, s, d]
  - the id is a unique identifier for that car
  - the x, y values are in global map coordinates
  - the vx, vy values are the velocity components, also in reference to the global map
  - s and d are the Frenet coordinates for that car


- the vx, vy values can be useful for predicting where the cars will be in the future
- for instance, if you were to assume that the tracked car kept moving along the road, then its future predicted Frenet s value will be its current s value plus its (transformed) total velocity (m/s) multiplied by the time elapsed into the future (s)


#### Changing Lanes

<img src="resources/changing_lanes.gif"/>

- the last consideration is how to create paths that can smoothly changes lanes
- any time the ego vehicle approaches a car in front of it that is moving slower than the speed limit, the ego vehicle should consider changing lanes
- the car should only change lanes if such a change would be safe, and also if the lane change would help it move through the flow of traffic better


- for safety, a lane change path should optimize the distance away from other traffic
- for comfort, a lane change path should also result in low acceleration and jerk
- the acceleration and jerk part can be solved from linear equations for s and d functions
- examples of this can be found in the Trajectory Generation quizzes entitled, "Quintic Polynomial Solver" and "Polynomial Trajectory"
- the provided Eigen-3.3 library can solve such linear equations
- the `getXY` helper function can transform (s,d) points to (x,y) points for the returned path

## Project Q&A

- https://youtu.be/7sI3VHFPP0w

## Project instructions

- in this project, your goal is to design a path planner that is able to create smooth, safe paths for the car to follow along a 3 lane highway with traffic
- a successful path planner will be able to keep inside its lane, avoid hitting other cars, and pass slower moving traffic all by using localization, sensor fusion, and map data


- you can find the project repository [here](https://github.com/udacity/CarND-Path-Planning-Project)


- in order to build and compile the code, from within the main repository directory, you can:
```shell
mkdir build && cd build  # to create and enter the build directory
cmake .. && make  # to compile your project
./path_planning  # to run your code
```
- important: you need to open a terminal before attempting to run the simulator